In [1]:
import pandas as pd


In [37]:
df = pd.read_csv('jobOffers.csv', index_col=0)

In [39]:
df.head()


,jobTitles,company,salaryRange,Bonus,category+progLang,seniority,workLanguage,location,remote
0,Java Software Developer,Intempt Technologies,23.2k - 34.9k CZK,NaN,"Backend, java",Junior,Work language:angličtina,NaN,Na dálku
1,C++ Developer,Sourcein,70k - 140k CZK,NaN,"Backend, C#","Mid, Senior",Work language:česky,"Prague, Altajská 1568/4 (Po pandemi)• Prague,...",Covid-time na dálku
2,C# Developer,Sourcein,70k - 140k CZK,NaN,"Backend, C#","Mid, Senior",Work language:česky,"Prague, Altajská 1568/4 (Po pandemi)• Prague,...",Covid-time na dálku
3,Java developer u Medvědů,Coding Bear,60k - 120k CZK,NaN,"Backend, java","Mid, Senior",Work language:česky,"Praha, Vítkova 241/10 • Praha, Vítkova 241/10",NaN
4,Senior Backend Developer,DX Heroes,60k - 100k CZK,NaN,"Backend, node","Mid, Senior","Work language:česky, angličtina","Praha, Rohanské nábř. 19 • Remote• Praha, Roh...",Na dálku


In [24]:
#salary
df = df.drop(df[df['salaryRange'] == 'neplacená stáž'].index)
df['salLower'] = df['salaryRange'].str.split(" - ").str[0].str.split("  ").str[0]
df['salUpper'] = df['salaryRange'].str.split(" - ").str[1].str.split("  ").str[0]

df['salLower'] = df['salLower'].replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)
df['salUpper'].fillna(df['salLower'], inplace = True)
df['salUpper'] = df['salUpper'].replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)

df['salAvg'] = (df['salUpper'] + df['salLower'])/2


In [25]:
#category and main programming language
df['category'] = df['category+progLang'].str.split(", ").str[0]
df['mainProgLang'] = df['category+progLang'].str.split(", ").str[1]
df.head()

,jobTitles,company,salaryRange,Bonus,category+progLang,seniority,workLanguage,location,remote,salLower,salUpper,salAvg,category,mainProgLang
0,Java Software Developer,Intempt Technologies,23.2k - 34.9k CZK,NaN,"Backend, java",Junior,Work language:angličtina,NaN,Na dálku,23200,34900,29050.0,Backend,java
1,C++ Developer,Sourcein,70k - 140k CZK,NaN,"Backend, C#","Mid, Senior",Work language:česky,"Prague, Altajská 1568/4 (Po pandemi)• Prague,...",Covid-time na dálku,70000,140000,105000.0,Backend,C#
2,C# Developer,Sourcein,70k - 140k CZK,NaN,"Backend, C#","Mid, Senior",Work language:česky,"Prague, Altajská 1568/4 (Po pandemi)• Prague,...",Covid-time na dálku,70000,140000,105000.0,Backend,C#
3,Java developer u Medvědů,Coding Bear,60k - 120k CZK,NaN,"Backend, java","Mid, Senior",Work language:česky,"Praha, Vítkova 241/10 • Praha, Vítkova 241/10",NaN,60000,120000,90000.0,Backend,java
4,Senior Backend Developer,DX Heroes,60k - 100k CZK,NaN,"Backend, node","Mid, Senior","Work language:česky, angličtina","Praha, Rohanské nábř. 19 • Remote• Praha, Roh...",Na dálku,60000,100000,80000.0,Backend,node


In [71]:
#seniority
df.groupby(['seniority']).size()
# adjusted seniority column
df["seniorityAdj"] = df["seniority"].str.strip().replace({'Trainee, Junior': 'Junior', 'Trainee': 'Junior','Senior, Expert':'Senior',
                                                          'Expert':'Senior', 'Mid, Senior':'Mid Advanced'})
df.groupby(['seniorityAdj']).size()     

seniorityAdj
Junior           70
Mid             190
Mid Advanced    243
Senior          214
dtype: int64

In [96]:
#Work language
df['language'] = df['workLanguage'].str.split(':').str[1]
df['language'] = df['language'].str.split(',')

df.groupby(['workLanguage']).size()
langList = ['angličtina', 'slovensky', 'česky' ]
for lang in langList:
    df[lang] = df['workLanguage'].str.contains(lang)

#['angličtina', 'slovensky', 'česky' ]
df['česky'].sum()

165

In [67]:
counts = df.groupby(['mainProgLang']).size().sort_values(ascending=False)
print(counts[counts>20])

mainProgLang
java          111
python         79
javascript     74
.net           58
react          55
php            44
dtype: int64


In [30]:
df['location'].str.contains('Praha|Prague')

0      True
1      True
2      True
3      True
4      True
       ... 
692     NaN
693    True
694     NaN
695     NaN
696     NaN
Name: location, Length: 697, dtype: object